In [1]:
%%html
<style>
    .jp-Cell {max-width: 1024px !important; margin: auto}
    .jp-Cell-inputWrapper {max-width: 1024px !important; margin: auto}
    .jp-MarkdownOutput p {text-align: justify;}
    .jupyter-matplotlib-figure {margin: auto;}
</style>

In [2]:
%matplotlib widget
from pyvisco import inter
GUI = inter.Control()

# PYVISCO - Prony series identification for linear viscoelastic material models
Author: Martin Springer | martin.springer@nrel.gov
***

## Overview
The mechanical response of linear viscoelastic materials is often described with Generalized Maxwell models. The necessary material model parameters are typically identified by fitting a Prony series to the experimental measurement data in either the frequency-domain (via Dynamic Mechanical Thermal Analysis) or time-domain (via relaxation measurements). Pyvisco performs the necessary data processing of the experimental measurements, mathematical operations, and curve-fitting routines to identify the Prony series parameters. The experimental data can be provided as raw measurement sets at different temperatures or as pre-processed master curves.
* If raw measurement data are provided, the time-temperature superposition principle is applied to create a master curve and obtain the shift functions prior to the Prony series parameters identification. 
* If master curves are provided, the shift procedure can be skipped, and the Prony series parameters identified directly. 
An optional minimization routine is provided to reduce the number of Prony elements. This routine is helpful for Finite Element simulations where reducing the computational complexity of the linear viscoelastic material models can shorten the simulation time.

In [3]:
display(GUI.b_theory)
display(GUI.out_theory)

ToggleButton(value=False, description='Click here for more details!', layout=Layout(width='200px'))

HTMLMath(value='')

***
## Parameter identification

### Specify measurement type and upload input data

In this section the measurement type of the input data is specified and the experimental data are uploaded. A set of example input files can be downloaded here: [Example input files](https://github.com/NREL/pyvisco/raw/main/examples/examples.zip)

#### Conventions 
Each input file needs to consist of two header rows. The first row indicates the column variables and the second row the corresponding units. The conventions used in this notebook are summarized below. Tensile moduli are denoted as $E$ and shear moduli  are denoted as $G$. Only the tensile moduli are summarized in the table below. For shear modulus data, replace `E` with `G`, e.g. `E_relax` -> `G_relax`.

| Physical quantity        | Symbol               | Variable   | Unit   |
| :---------------------   | :-------------:      | :--------- | :----: |
| Relaxation modulus:      | $E(t)$               | `E_relax`  | `[Pa, kPa, MPa, GPa]`    |
| Storage modulus:         | $E'(\omega)$         | `E_stor`   | `[Pa, kPa, MPa, GPa]`    |
| Loss modulus:            | $E''(\omega)$        | `E_loss`   | `[Pa, kPa, MPa, GPa]`    |
| Complex modulus:         | $\lvert E^{*}\rvert$ | `E_comp`   | `[Pa, kPa, MPa, GPa]`    |
| Loss factor:             | $\tan(\delta)$       | `tan_del`  | `-`                      |
| Instantaneous modulus:   | $E_0$                | `E_0`      | `[Pa, kPa, MPa, GPa]`    |
| Equilibrium modulus:     | $E_{inf}$            | `E_inf`    | `[Pa, kPa, MPa, GPa]`    |
| Angular frequency:       | $\omega$             | `omega`    | `rad/s`                  |
| Frequency:               | $f$                  | `f`        | `Hz`                     |
| Time:                    | $t$                  | `t`        | `s`                      |
| Temperature:             | $\theta$             | `T`        | `°C`                     |
| Relaxation times:        | $\tau_i$             | `tau_i`    | `s`                      |
| Relaxation moduli:       | $E_i$                | `E_i`      | `[Pa, kPa, MPa, GPa]`    |
| Norm. relaxation moduli: | $\alpha_i$           | `alpha_i`  | `-`                      |
| Shift factor:            | $$\log (a_{T})  $$   | `log_aT`   | `-`                      |

***
#### Domain
The Prony series parameters can be either fitted from measurement data of Dynamic Mechanical Thermal Analysis (DMTA) in the frequency domain (freq) or from relaxation experiments in the time domain (time). 

#### Loading
Measurement data can be provided from either tensile or shear experiments. Tensile moduli are denoted as $E$ and shear moduli  are denoted as $G$.

#### Instrument
* **Eplexor:** DMTA measurements conducted with a Netzsch Gabo DMA EPLEXOR can be directly uploaded as Excel files. Use the `Excel Export!` feature of the Eplexor software with the default template to create the input files. 
* **user:** Comma-separated values (csv) files are used for measurements conducted with other instruments. Prepare the input files in accordance to the performed measurement. The table below shows example headers, where the first row indicates the column quanitity and the second row the corresponding unit.

| Domain               | Tensile              | shear               | 
| :-----------         | :-------------       | :---------          |
|**Frequency domain:** | `f, E_stor, E_loss`  | `f, G_stor, G_loss` |
|                      | `Hz, MPa, MPa`       | `Hz, GPa, GPa`      |
|**Time domain:**      | `t, E_relax`         | `t, G_relax`        |
|                      | `s, MPa`             | `s, GPa`            |


#### Type

Either **raw** data containing measurements at different temperatures or ready to fit **master** curves can be uploaded. 

Input files of **raw** measurement data need to specify the individual temperature sets of the performed experimental characterization:
* **Eplexor:** The notebook identifies the corresponding temperature sets automatically (only available in the frequency domain).
* **user:** Two additional columns need to be included in the input file. One column describing the temperature `T` of the measurement point and a second column `Set` to identify the corresponding measurement set of the data point (e.g. `f, E_stor, E_loss, T, Set`). All measurement points at the same temperature level are marked with the same number, e.g. 0 for the first measurement set. The first measurement set (0) represents the coldest temperature followed by the second set (1) at the next higher temperature level and so forth (see the provided [example input file](https://github.com/NREL/pyvisco/blob/main/examples/time_user_raw.csv) for further details).

In [4]:
display(GUI.w_inp_gen)

***
#### Optional shift factor upload

Already existing shift factors can be uploaded as a csv file with the header=`T, log_aT` and units=`C, -`, where `T` is the temperature level of the measurement set in Celsius and `log_aT` is the base 10 logarithm of the shift factor, $\log(a_T)$.

* **master:** Uploading the shift factors allows for the calculation of polynomial (D1 to D4) shift functions and the Williams–Landel–Ferry (WLF) shift function, but is not required for the Prony series estimation. 
> _**Note**_: If a master curve from the Eplexor software is provided, the default behavior of the notebook is to use the WLF shift function from the Eplexor software. However, in the time-temperature superpostion section, a checkbox is provided to overwrite the WLF fit of the Eplexor software and conduct another WLF fit with the algorithm in this notebook.

* **raw:** The shift factors can either be directly determined for the desired reference temperature in the time-temperature superposition section (no upload necessary) or user-specified shift factors can be uploaded to be used in the creation of the master curve. 

In [5]:
display(GUI.w_inp_shift)

***
#### Reference temperature 

Temperature chosen to contruct the master curve through application of the time-temperature superposition principle $[\log(a_T) = 0]$.

In [6]:
display(GUI.w_RefT)

***
#### Check uploaded data

In [7]:
display(GUI.w_check_inp)

***
### Time-temperature superposition (shift functions)

This section allows the calculation of shift factors from raw input data to create a master curve. The shift factors are then used to fit polynomial and WLF shift functions.

#### Shift factors $\log(a_{T})$ - Create master curve from raw input

The time-temperature superposition principle is applied to create a master curve from the individual  measurement sets at different temperature levels. 

> _**Note**_:This subsection only applies to raw measurement data. If a master curve was uploaded, procede to the next step. 

  * **user shift factors uploaded:** The provided shift factors will be used to create the master curve from the raw measurement sets (_**Note**_: the `fit and overwrite provided shift factors` checkbox allows you to overwrite the uploaded user shift factors and fit new ones). 
  * **No user shift factors uploaded:** The measurement sets from the raw input file are used to estimate the shift factors and create a master curve. Measurement sets below the desired reference temperatures are shifted to lower frequencies (longer time periods), whereas measurement sets at temperatures higher than the reference temperature are shifted to higher frequencies (shorter time periods). The `manually adjust shift factors` checkbox allows you to modify the obtained shift factors manually. (_**Note**_: In the frequency domain, only the storage modulus input data are considered to create the master curve from the raw input data. The shift factors obtained from the storage modulus master curve are then used to create the loss modulus master curve.)

In [8]:
display(GUI.w_aT)

***
#### Shift functions (WLF & Polynomial degree 1 to 4)

If shift factors are available, the WLF shift function and polynomial functions of degree 1 to 4 can be fitted and plotted below. (_**Note**_: If the WLF shift functions was already provided by the Eplexor software, the checkbox below let's you overwrite the WLF fit of the Eplexor software with a WLF fit of this notebook.) 

> _**Note**_:This subsection only provides shift functions and is not required to perform the parameter identification of the Prony series.

In [9]:
display(GUI.w_shift)

***
### Estimate Prony series parameters

#### Pre-process (smooth) master curve

A moving median filter to remove outliers in the measurement data can be applied before the Prony series parameters are identified. The window size can be adjusted through the slider above the figure. A window size of 1 means that no filtering procedure is performed and the input data are fitted directly.

In [10]:
display(GUI.w_smooth)

***
#### Define the number and discretization of the Prony series

The number of Prony terms, $N$, needs to be defined before the parameter $\tau_i$ and $\alpha_i$ can be identified. The `default` behavior is to equally space one Prony term per decade along the logarithmic time axis, e.g., $\tau_i$ = [1E-1, 1E0, 1E1,...] (s). This discretization typically delivers accurate results for engineering applications. 
> _**Note:**_ The fine discretization can be computationally heavy for using the viscoelastic material models in Finite Element simulations. Hence, the default discretization can be modified by either using the optimization routine provided below or by manually defining the number of Prony terms (`manual`). Here, the user can decide whether to round the lowest and highest relaxation times, $\tau_i$, to the nearest base 10 number within the measurement window `round` or to use the exact minimum and maximum values of the experimental data for the relaxation times `exact`. 

In [11]:
display(GUI.w_dis)

***
#### Curve fitting

Two different curve fitting routines for the Prony series parameters are employed and are dependent on the domain of the input data:

* **Frequency domain**: A generalized collocation method using stiffness matrices is used as described in [Kraus, M. A., and M. Niederwald. Eur J Eng Mech 37.1 (2017): 82-106](https://journals.ub.ovgu.de/index.php/techmech/article/view/600). This methods utilizes both the storage and loss modulus master curves to estimate the Prony series parameters.
   
   
* **Time domain**: A least-squares minimization is performed using the L-BFGS-B method from the scipy package. The implementation is similar to the optimization problem described by [Barrientos, E., Pelayo, F., Noriega, Á. et al. Mech Time-Depend Mater 23, 193–206 (2019)](https://doi.org/10.1007/s11043-018-9394-z) for a homogenous distribution of discrete times. 

In [12]:
display(GUI.w_out_fit_prony)

***
#### Generalized Maxwell model

The fitted Prony series parameters in combination with the Generalized Maxwell model can be used to calculate the linear viscoelastic material parameters in both the time and frequency domain. 

In [13]:
display(GUI.w_out_GMaxw)

***
### Optional: Minimize number of Prony elements (for Finite Element simulations)

The Generalized Maxwell model with a high number of Prony terms can be computationally expensive. Especially, when used in combination with numerical frameworks as the Finite Element Method. Reducing the number of Prony elements will decrease the accuracy of the linear viscoelastic material model, but can help to speed up subsequent numerical simulations.

We provide a simple routine to create an additional Generalized Maxwell model with a reduced number of Prony elements. The routine starts with the number of Prony terms specified above and subsequently reduces the number of terms. A least squares minimization is performed to fit the reduced term Prony parameters. The least squares residual is used to suggest an optimal number of Prony terms for subsequent FEM simulations ($R_{opt}^2 \approx$ $1.5 R_0^2$). However, the user can change this default setting by selecting a different number of Prony terms below.

> **_Note:_** This routine is computationally more demanding and can take a few minutes to complete. The runtime depends on the initial number of Prony elements and the number of data points in the measurement sets.

In [14]:
display(GUI.w_out_fit_min)

***
## Download results

A zip archive including the identified Prony series parameters, (shift factors and shift functions), results of the Generalized Maxwell model, and figures can be dowloaded below.

In [15]:
#display(GUI.db_zip)
display(GUI.w_out_down)

***
## Start over!
Clear all input data and reload an empty notebook.

In [16]:
display(GUI.w_reload)